# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Введение

### Описание проекта

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо помочь добывающей компании «ГлавРосГосНефть» определиться где бурить новую скважину.

### Цель проекта

Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализиовать возможную прибыль и риски техникой Bootstrap.

### Описание исследования

- В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
- Строят модель для предсказания объёма запасов в новых скважинах;
- Выбирают скважины с самыми высокими оценками значений;
- Определяют регион с максимальной суммарной прибылью отобранных скважин.

### Задачи исследования

- Обучить и проверьте модель для каждого региона;
- Подготовить данные к расчёту прибыли;
- Написать функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели;
- Посчитайте риски и прибыль для каждого региона.

Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### Исходные данные

Данные геологоразведки трёх регионов: 

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

###  Используемые библиотеки

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

###  Используемые константы

In [2]:
TEST_SIZE = 0.25
RANDOM_STATE = 42
state = np.random.RandomState(12345)

point = 500 # кол-во точен при разведки региона

BEST_POINT = 200 #  кол-во выбранных лучши точек, которые выбирают с помощью машинного обучения

BUDGET = 10000000000 # бюджет на разработку скважин в регионе

INCOME_UNIT = 450000 # доход с каждой единицы продукта.

threshold = 0.025 # порог безубыточности

BOOTSTRAP_COINT = 1000 # Bootstrap с 1000 выборок

### Свои функции использумые в проекте

**Загрузка данных**

In [3]:
def load_df(file, index='id'):
    pth1 = f'/datasets/{file}'
    pth2 = f'{file}'

    if os.path.exists(pth1):
        df = pd.read_csv(pth1, index_col=index, na_values=np.nan)
    elif os.path.exists(pth2):
        df = pd.read_csv(pth2, index_col=index, na_values=np.nan)
    else:
        print('Что-то пошло не так')
    
    return df

**Функция вывода данных и их описания**

In [4]:
def get_data_info(df):
    display(df.head())
    display(HTML('<br>'))
    display(HTML('<font color="gray"><b>Информации о датафрейме</b></font>'))
    df.info()
    display(HTML('<br>'))
    display(HTML('<font color="gray"><b>Описание данных</b></font>'))
    display(df.describe().T)
    display(HTML('<br>'))
    display(HTML('<font color="gray"><b>Количество пропусков</b></font>'))
    display(df.isna().sum())
    display(HTML('<br>'))
    display(HTML('<font color="gray"><b>Количество дубликатов</b></font>'))
    display(df.duplicated().sum())
    display(HTML('<br>'))

**Разделение данных**

In [5]:
def df_split(df, target):
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop([target], axis=1),
        df[target],
        test_size = TEST_SIZE, 
        random_state = RANDOM_STATE)
    
    return X_train, X_test, y_train, y_test

**Обучение модели**

In [6]:
def lg_model(X_train, y_train, X_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    
    return predict

**Функция для расчёта прибыли**

In [7]:
def revenue(target, probabilities, count=BEST_POINT, income_unit=INCOME_UNIT, budget=BUDGET):
    probs_sorted = probabilities.sort_values(ascending=False) # cортируем по убыванию предсказания
    selected = target[probs_sorted.index][:count]  # выбираем топ точки по сырью на основе предсказаний
    return round(selected.sum() * income_unit - budget, 2) # рассчитываем и возвращаем прибыль

**Расчет рисков и прибыли**

In [8]:
def bootstrap(target, probabilities, bootstrap_coint=BOOTSTRAP_COINT):
    
    values = []
    
    for i in range(bootstrap_coint):
        target_subsample = target.sample(n=point, replace=True, random_state=state) # получаем сэмпл из таргета из 500 точек
        # по индексам сэмпла таргета получаем сэмпл из предсказанных значений
        probs_subsample = probabilities[target_subsample.index]

        profit = revenue(target_subsample, probs_subsample) # рассчитываем прибыль
        values.append(profit) # и сохраняем значение в список

    values = pd.Series(values)
    lower = round(values.quantile(threshold), 2)
    upper = round(values.quantile(1 - threshold), 2)

    mean = round(values.mean(), 2)

    risk = round((values < 0).mean() * 100, 2)

    display(f'Средняя прибыль: {"{0:,}".format(mean).replace(",", " ")}')
    display(f'95%-й доверительный интервал от {"{0:,}".format(lower).replace(",", " ")}'
           f' до {"{0:,}".format(upper).replace(",", " ")}')
    display(f'Риск убытков {risk }%')
    

## Загрузка и подготовка данных

### Загрузка данных

In [9]:
# Тренировочная выборка:
geo_data_0 = load_df('geo_data_0.csv')

# Входные признаки тестовой выборки:
geo_data_1 = load_df('geo_data_1.csv')

# Целевой признак тестовой выборки:
geo_data_2 = load_df('geo_data_2.csv')

### Общая информация о датафреймах

In [10]:
dict_df = {'geo_data_0': geo_data_0, 
           'geo_data_1': geo_data_1, 
           'geo_data_2': geo_data_2}

In [11]:
for k, v in dict_df.items():
    display(HTML(f'<font color="navy"><b>Вывод информации о данных {k}</b></font>'))
    get_data_info(v)

,f0,f1,f2,product
id,,,,
txEyH,0.705745,-0.497823,1.221170,105.280062
2acmU,1.334711,-0.340164,4.365080,73.037750
409Wp,1.022732,0.151990,1.419926,85.265647
iJLyR,-0.032172,0.139033,2.978566,168.620776
Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, txEyH to 1CWhH
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


f0         0
f1         0
f2         0
product    0
dtype: int64

0

...

,f0,f1,f2,product
id,,,,
fwXo0,-1.146987,0.963328,-0.828965,27.758673
WJtFt,0.262778,0.269839,-2.530187,56.069697
ovLUW,0.194587,0.289035,-5.586433,62.871910
q6cA6,2.236060,-0.553760,0.930038,114.572842
WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, fwXo0 to V9kWn
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f0       100000 non-null  float64
 1   f1       100000 non-null  float64
 2   f2       100000 non-null  float64
 3   product  100000 non-null  float64
dtypes: float64(4)
memory usage: 3.8+ MB


,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.002023,1.732045,-8.760004,-1.162288,0.009424,1.158535,7.238262
f1,100000.0,-0.002081,1.730417,-7.084020,-1.174820,-0.009482,1.163678,7.844801
f2,100000.0,2.495128,3.473445,-11.970335,0.130359,2.484236,4.858794,16.739402
product,100000.0,95.000000,44.749921,0.000000,59.450441,94.925613,130.595027,190.029838


f0         0
f1         0
f2         0
product    0
dtype: int64

0

### Вывод

Выводы по загруженным данным:

- `id` - перевел в индекс, так как при обучении нам этот признак не понадобится.
- Что означают признаки `f0`, `f1`, `f2` заказчиком скрыты.
    
Датафрейм `geo_data_0`:
- Типы данных соответствуют содержимому;
- Общее описание данных не вызывает вопросов;
- Cодержит 100000 строк;
- Пропуски отсутствуют;
- Дубликаты осутствуют.

Датафрейм `geo_data_1`:
- Типы данных соответствуют содержимому;
- Общее описание данных не вызывает вопросов;
- Cодержит 100000 строк;
- Пропуски отсутствуют;
- Дубликаты осутствуют.

Датафрейм `geo_data_2`:
- Типы данных соответствуют содержимому;
- Присутствет значительная разность среднего значения и медианы по `f0` и `f1`;
- Cодержит 100000 строк;
- Пропуски отсутствуют;
- Дубликаты осутствуют.




## Обучение и проверка модели

### Разделение данных

на обучающую и валидационную выборки в соотношении 75:25.

In [12]:
X_train_0, X_test_0, y_train_0, y_test_0 = df_split(geo_data_0, 'product')

In [13]:
X_train_1, X_test_1, y_train_1, y_test_1 = df_split(geo_data_1, 'product')

In [14]:
X_train_2, X_test_2, y_train_2, y_test_2 = df_split(geo_data_2, 'product')

### Обучение модели и предсказания на валидационной выборке

In [15]:
predict_0 = lg_model(X_train_0, y_train_0, X_test_0)

In [16]:
predict_1 = lg_model(X_train_1, y_train_1, X_test_1)

In [17]:
predict_2 = lg_model(X_train_2, y_train_2, X_test_2)

### Cредний запас предсказанного сырья и RMSE модели

**Регион 1**

In [18]:
display(f'Cредний запас предсказанного сырья - {predict_0.mean()}')
display(f'RMSE - {mean_squared_error(y_test_0, predict_0)**0.5}')

'Cредний запас предсказанного сырья - 92.39879990657768'

'RMSE - 37.75660035026169'

**Регион 2**

In [19]:
display(f'Cредний запас предсказанного сырья - {predict_1.mean()}')
display(f'RMSE - {mean_squared_error(y_test_1, predict_1)**0.5}')

'Cредний запас предсказанного сырья - 68.71287803913762'

'RMSE - 0.8902801001028859'

**Регион 3**

In [20]:
display(f'Cредний запас предсказанного сырья - {predict_2.mean()}')
display(f'RMSE - {mean_squared_error(y_test_2, predict_2)**0.5}')

'Cредний запас предсказанного сырья - 94.77102387765939'

'RMSE - 40.145872311342174'

**Полученным предсказаниям присвоим индексы (id скважины)**

In [21]:
predict_0 = pd.Series(predict_0, index=y_test_0.index)
predict_1 = pd.Series(predict_1, index=y_test_1.index)
predict_2 = pd.Series(predict_2, index=y_test_2.index)

### Вывод

Среднеквадратичная ошибка (RMSE) модели регионов 1 и 3 достаточно высоская, 37.76 и 40.15, соответственно. Это говорит о том, что средний запас предсказанного сырья может быть недостаточно точным, по сравнению с моделью региона 2, у которой RMSE значительно ниже и составляет 0,89.

## Подготовка к расчёту прибыли

### Достаточный объём сырья для безубыточной разработки

Чтобы расчитать достаточный объем для безубыточной разработки требуется общий бюджет разделить на количество точек умноженное на доход с еденицы продукта.

In [22]:
sufficient_volume = BUDGET / (BEST_POINT * INCOME_UNIT)
sufficient_volume

111.11111111111111

### Cравнение полученного объём сырья со средним запасом в каждом регионе

In [23]:
geo_data_0['product'].mean() - sufficient_volume

-18.6111111111111

In [24]:
geo_data_1['product'].mean() - sufficient_volume

-42.2861111111111

In [25]:
geo_data_2['product'].mean() - sufficient_volume

-16.11111111111107

### Вывод

Достаточный объем для безубыточной разработки составляет 111.11 ед. продукта.
Что на 18,61 больше среднего значения региона 1, на 42.29 среднего значения региона 2 и 16.11 региона 3.

## Расчёт прибыли и рисков 

### Расчёт прибыли

In [26]:
revenue(y_test_0, predict_0)

3359141114.46

In [27]:
revenue(y_test_1, predict_1)

2415086696.68

In [28]:
revenue(y_test_2, predict_2)

2598571759.37

#### Вывод

Все регионы получись безубыточными:
- Прибыль региона 1 составляет 3,4 млрд руб.
- Прибыль региона 2 составляет 2,4 млрд руб.
- Прибыль региона 3 составляет 2,6 млрд руб.

### Расчет рисков и прибыли техникой Bootstrap

In [29]:
bootstrap(y_test_0, predict_0)

'Средняя прибыль: 435 933 772.14'

'95%-й доверительный интервал от -116 231 612.78 до 966 504 180.71'

'Риск убытков 6.1%'

In [30]:
bootstrap(y_test_1, predict_1)

'Средняя прибыль: 498 569 016.92'

'95%-й доверительный интервал от 75 315 573.09 до 911 503 695.02'

'Риск убытков 1.0%'

In [31]:
bootstrap(y_test_2, predict_2)

'Средняя прибыль: 412 672 898.99'

'95%-й доверительный интервал от -158 939 515.77 до 978 191 201.7'

'Риск убытков 7.0%'

### Вывод

Полученные величины по регионам:

**Регион 1**

- Средняя прибыль: 435 933 772.14;
- 95%-й доверительный интервал от -116 231 612.78 до 966 504 180.71;
- Риск убытков 6.1%.

**Регион 2**

- Средняя прибыль: 498 569 016.92;
- 95%-й доверительный интервал от 75 315 573.09 до 911 503 695.02;
- Риск убытков 1.0%.

**Регион 3**

- Средняя прибыль: 412 672 898.99;
- 95%-й доверительный интервал от -158 939 515.77 до 978 191 201.7;
- Риск убытков 7.0%.

Основываясь на полученных данных предположу, что лучше выбрать Регион 2 для разработки скважин, так как риск безубыточной разботки составляет 99%

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован